# Lisette Werk - Aangepast door Domenico 

In [3]:
using StaticArrays
using ForwardDiff
import Gmsh: gmsh

In [4]:
# a point in 3D is a tuple of 3 coordinates 
# we here introduce static vectors that appear to be vital to reduce the number of allocations 
const Point3D = SVector{3,Float64};
const Triangle = SVector{7,Point3D};

In [5]:
path = "C:\\Users\\bruinlmd\\Documents\\Afstuderen\\MoM\\" 

#include(path * "Visualize_results.jl")
#include(path * "Create_mesh.jl")
#include(path * "Solve_step.jl")
#include(path * "Assembly.jl")

"C:\\Users\\bruinlmd\\Documents\\Afstuderen\\MoM\\"

In [6]:
function createGeometryAndMesh_rectangular_plate(x_max, y_max, lc)
    
    # Clear all models and create a new one
    gmsh.clear()

    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(x_max, 0,  0, lc, 2)
    gmsh.model.geo.addPoint(x_max, y_max, 0, lc, 3)
    gmsh.model.geo.addPoint(0, y_max, 0, lc, 4)
    
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(3, 2, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 1, 4)
    
    gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(2)
end

function checkForEvent()
    action = gmsh.onelab.getString("ONELAB/Action")
    if length(action) > 0 && action[1] == "check"
        gmsh.onelab.setString("ONELAB/Action", [""])
        createGeometryAndMesh()
        gmsh.graphics.draw()
    end
    return true
end

checkForEvent (generic function with 1 method)

In [7]:
# t: half thickness of the plate - see z-coordinate definition 
function process_mesh(nelements, element_connectivity, xnode, ynode, t)
        
    x0=0.; y0=0.; z0=0.; x1=1.; y1=1.;z1=1.;
    # six vertices of the elements (?)
    v1 = Point3D(x0,y0,z0)
    v2 = Point3D(x1,y0,z0)
    v3 = Point3D(x1,y1,z0)
    v4 = Point3D(x0,y1,z0)
    v5 = Point3D(x0,y0,z1)
    v6 = Point3D(x1,y0,z1)
    
    f1 = Triangle(v1, v2, v3, v1, v1, v1, v1)
    f2 = Triangle(v1, v3, v4, v1, v1, v1, v1)
    f3 = Triangle(v1, v2, v3, v1, v1, v1, v1)
    f4 = Triangle(v1, v2, v3, v1, v1, v1, v1)
    f5 = Triangle(v1, v2, v6, v1, v1, v1, v1)
    f6 = Triangle(v1, v6, v5, v1, v1, v1, v1)
    f7 = Triangle(v1, v2, v3, v1, v1, v1, v1)
    f8 = Triangle(v1, v2, v3, v1, v1, v1, v1)
        
    # define the vertex type and face type 
    vertextype = typeof(@SVector [v1,v2,v3,v4,v5,v6]) 
    facetype = typeof(@SVector [f1,f2,f3,f4,f5,f6,f7,f8])
    
    num_boxes = nelements
    vertices_list = Vector{vertextype}(undef,num_boxes)
    faces_list = Vector{facetype}(undef,num_boxes)
    
    for element_id in 1:nelements
    
        #....retrieve global numbering of the local nodes of the current element
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
    
        if (false)
          println("on element ", element_id, " node-1 has global number ", node1_id, " and the (x,y)-coordinate is ",  xnode[node1_id] ,", ",ynode[node1_id])
          println("on element ", element_id, " node-2 has global number ", node2_id, " and the (x,y)-coordinate is ",  xnode[node2_id] ,", ",ynode[node2_id])
          println("on element ", element_id, " node-3 has global number ", node3_id, " and the (x,y)-coordinate is ",  xnode[node3_id] ,", ",ynode[node3_id])
          println(" ")
        end 
    
        node11_x = xnode[node1_id] ; node11_y = ynode[node1_id]; node11_z = -t
        node12_x = xnode[node1_id] ; node12_y = ynode[node1_id]; node12_z = t
        
        node21_x = xnode[node2_id] ; node21_y = ynode[node2_id]; node21_z = -t
        node22_x = xnode[node2_id] ; node22_y = ynode[node2_id]; node22_z = t
    
        node31_x = xnode[node3_id] ; node31_y = ynode[node3_id]; node31_z = -t
        node32_x = xnode[node3_id] ; node32_y = ynode[node3_id]; node32_z = t
    
        vertices = vertextype( Point3D(node11_x, node11_y, node11_z), Point3D(node12_x, node12_y, node12_z), Point3D(node21_x, node21_y, node21_z), Point3D(node22_x, node22_y, node22_z), Point3D(node31_x, node31_y, node31_z), Point3D(node32_x, node32_y, node32_z)) 
        vertices_list[element_id] = vertices

        v1, v2, v3, v4, v5, v6 = vertices
    
        r_k = find_center_element(vertices)
   
        faces = facetype(
                create_triangle_uniform(v1, v3, v5, r_k),
                create_triangle_uniform(v2, v4, v6, r_k),
                create_triangle_uniform(v1, v3, v2, r_k),
                create_triangle_uniform(v2, v3, v4, r_k),
                create_triangle_uniform(v3, v5, v4, r_k),
                create_triangle_uniform(v4, v5, v6, r_k),
                create_triangle_uniform(v1, v5, v2, r_k),
                create_triangle_uniform(v2, v5, v6, r_k))
        
        faces_list[element_id] = faces
    
    end 
    
    return vertices_list, faces_list
end

process_mesh (generic function with 1 method)

A mesh is created with four elements. Then, we make a list of the vertices and the faces per element. 

In [8]:
x_max = 1
y_max = 1
lc = 1

gmsh.initialize()
createGeometryAndMesh_rectangular_plate(x_max, y_max, lc)

t = 0.001

element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2);
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()

tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
sorted = sortslices(tosort , dims = 1);
node_ids = sorted[:,1]
xnode = sorted[:,2]
ynode = sorted[:,3]

nelements = length(element_ids[1]);
nNodes = length(node_coord[1]);

vertices_list, faces_list = process_mesh(nelements, element_connectivity, xnode, ynode, t);


Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000164208s, CPU 0.000156s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.000131208s, CPU 0.000104s)
Info    : 5 nodes 12 elements


LoadError: UndefVarError: find_center_element not defined

In the following code, the function f_vector is the function over which we want to integrate. The function curl_of_function takes the curl of an abritrary function.

In [ ]:
function f_vector(r_accent, r, M)
    
    teller = cross(M, (r - r_accent))
    noemer = norm(r - r_accent)^3
    result = teller/noemer
    return @SVector[result[1], result[2], result[3]]
   
end

function curl_of_function(f, r)
    y = similar(r)
    cfg = ForwardDiff.JacobianConfig(f, y, r, ForwardDiff.Chunk{3}())
    Jac = Matrix{Float64}(undef, length(y), length(r))
    ForwardDiff.jacobian!(Jac, f, y, r, cfg)
    curl_x = Jac[3,2] - Jac[2,3]
    curl_y = Jac[1,3] - Jac[3,1]
    curl_z = Jac[2,1] - Jac[1,2]
    
    return @SVector[curl_x, curl_y, curl_z] 
end

The following functions takes as input the vertices of the prism. It transforms the variables such that the integration in the x- and y-direction is from (0,0) (1,1). 

In [ ]:
function integrate_over_prism_using_HCubature(f, M, r1_int, r2_int, r3_int, t, r, r_tol)

    g(u,v) = (1-u)*r1_int[1] + u* ((1 - v) * r2_int[1] + v * r3_int[1])
    h(u,v) = (1-u)*r1_int[2] + u* ((1 - v) * r2_int[2] + v * r3_int[2])

    dxdu(u,v) = -r1_int[1] + (1-v)*r2_int[1] + v*r3_int[1]
    dxdv(u,v) = -u * r2_int[1] + u * r3_int[1]

    dydu(u,v) = -r1_int[2] + (1-v)*r2_int[2] + v*r3_int[2]
    dydv(u,v) = -u * r2_int[2] + u * r3_int[2]

    J(u,v) = abs(dxdu(u,v) * dydv(u,v) - dxdv(u,v) * dydu(u,v))

    integrand(u,v,z) = f(@SVector[g(u,v),h(u,v),z], r, M) * J(u,v)


    integrand_result = hcubature(x -> integrand(x[1], x[2], x[3]), (0.0, 0.0, -t), (1.0, 1.0, t), rtol = r_tol)[1]
    return @SVector[integrand_result[1],integrand_result[2],integrand_result[3]]
    
end

Since the singularity arise when the "self-term" is calculated, we first calculate the integral at the center of the first element and also integrating over the first element. As an example $\vec{M} = [1,0,0]$. 

In [ ]:
r_tol = 10^(-5)
M = @SVector[1,0,0]
vertices_ele1 = vertices_list[1]

#Define the vertices of the prism
r1_int = (vertices_ele1[1] + vertices_ele1[2])/2
r2_int = (vertices_ele1[3] + vertices_ele1[4])/2
r3_int = (vertices_ele1[5] + vertices_ele1[6])/2

#I want to evaluate the function at the center of the first element
r = find_center_element(vertices_list[1])

a(r) = integrate_over_prism_using_HCubature(f_vector, M, r1_int, r2_int, r3_int, t, r, r_tol)

function a_wrapper!(y, r)
    copyto!(y, a(r))
end

B = μ0_div_4π*curl_of_function( a_wrapper!, r)

In [ ]:
println(a(r))

Now, we calculate the integral at the center of the second element, but integrating over the first element.

In [ ]:
r_tol = 10^(-5)
M = @SVector[1,0,0]
vertices_ele1 = vertices_list[1]

#Define the vertices of the prism
r1_int = (vertices_ele1[1] + vertices_ele1[2])/2
r2_int = (vertices_ele1[3] + vertices_ele1[4])/2
r3_int = (vertices_ele1[5] + vertices_ele1[6])/2

#I want to evaluate the function at the center of the second element
r = find_center_element(vertices_list[2])

a(r) = integrate_over_prism_using_HCubature(f_vector, M, r1_int, r2_int, r3_int, t, r, r_tol)

function a_wrapper!(y, r)
    copyto!(y, a(r))
end

B = μ0_div_4π*curl_of_function( a_wrapper!, r)

In [ ]:
println(a(r))

We see that calculating the self-term gives a vector potential only containing NaN-values. 